Monografia - Parte 2

Especialización en Analítica y Ciencia de Datos

Universidad de Antioquia

Grupo 3. Melissa Ortega Alzate CC.1036964792

# Objetivo

# Resumen

# Librerías

In [8]:
# Data treatment
import numpy as np
import pandas as pd

# Figures
import matplotlib.pyplot as plt
import seaborn as sns

# Data preprocessing
from sklearn.model_selection import train_test_split

# Librerías de Deep Learning
from keras import layers, models

# Models metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# Funciones

In [9]:
# ROC curve function

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme( palette= "pastel", style= "ticks", rc= custom_params)

def plot_roc_curve(models: list, X_test, y_test):
    models_name = []
    for model in models:
        models_name.append(f'{model}'.split('(')[0])

    fig = plt.figure(figsize=(10, 6))
    entries = []

    for i in range(len(models)):
        model = models[i]
        probs = model.predict_proba(X_test)              # Obtener probabilidades de predicción del modelo
        probs = probs[:, 1]                              # Seleccionar la columna correspondiente a la probabilidad de clase positiva
        auc = roc_auc_score(y_test, probs)               # Calcular el AUC-ROC
        fpr, tpr, _ = roc_curve(y_test, probs)           # Calcular las tasas de FPR y TPR      
        plt.plot(fpr, tpr, label=f"{model.__class__.__name__} (AUC={auc:.2f})") # Graficar la curva ROC del modelo actual
        entries.append(( models_name[i], auc))
        #print("{:.2f}: {}".format(auc, model))            # Mostrar el AUC-ROC y el nombre del modelo

    df_roc = pd.DataFrame(entries, columns=['Model', 'AUC'])
    print(f" AUC de los modelos con orden descendente " +"\n" + "="*45)
    pd.options.display.max_colwidth = 100
    display(df_roc.sort_values(by='AUC', ascending= False))
    print("\n")

    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show();

# Desarrollo

### Cargar datos

In [10]:
# Cargar X desde el archivo .npy
X = np.load('X.npy')
X[0]

array([0.36078431, 0.37254902, 0.39215686, ..., 0.57254902, 0.45098039,
       0.32941176])

In [11]:
# Cargar y desde el archivo .csv
y = pd.read_csv('y.csv')['Salmonella']
y[0]

1

### Split train and test

In [12]:
# Dividir el DataFrame en los trenes de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y,  
                                                    train_size= 0.8, 
                                                    shuffle= True, 
                                                    random_state= 123, 
                                                    stratify= y)

# Imprimir el tamaño de los trenes de entrenamiento y prueba
print("Tamaño del tren de entrenamiento:", X_train.shape)
print("Tamaño de las etiquetas de entrenamiento:", y_train.shape)
print("-------")
print("Tamaño del tren de prueba:", X_test.shape)
print("Tamaño de las etiquetas de prueba:", y_test.shape)

Tamaño del tren de entrenamiento: (800, 150528)
Tamaño de las etiquetas de entrenamiento: (800,)
-------
Tamaño del tren de prueba: (200, 150528)
Tamaño de las etiquetas de prueba: (200,)


### Baseline

In [13]:
# Instanciar el modelo de tipo Sequential
model = models.Sequential()

# 2. Capa densa de activación con 512 neuronas y una activación de tipo relu
model.add(layers.Dense(512, activation='relu', input_shape = (224*224*3,)))

# 3. Capa adicional con 1 neuronas (porque tengo 1 posible salida) y una activación tipo sigmoid para clasificación binara
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 512)            │    77,070,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,071,361 (294.00 MB)

 Trainable params: 77,071,361 (294.00 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compilar el modelo
model.compile(loss='binary_crossentropy',
            optimizer = 'rmsprop',
            metrics =['accuracy'])

In [15]:
# Cambiar a 2 dimensiones
X_train = X_train.reshape(X_train.shape[0], 224*224*3)      
X_test = X_test.reshape(X_test.shape[0], 224*224*3)      

In [16]:
# Entrenar el modelo
model.fit(X_train, y_train, batch_size= 100, epochs=5)

Epoch 1/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 770ms/step - accuracy: 0.5385 - loss: 242.5381
Epoch 2/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 734ms/step - accuracy: 0.4894 - loss: 54.0599
Epoch 3/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 734ms/step - accuracy: 0.5177 - loss: 13.3236
Epoch 4/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 685ms/step - accuracy: 0.4705 - loss: 11.2272
Epoch 5/5
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 631ms/step - accuracy: 0.5377 - loss: 6.1939


In [17]:
# Evaluar rendimiento del modelo
model.evaluate(X_test, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5424 - loss: 8.1305


[8.848474502563477, 0.5049999952316284]

# Conclusiones

# Bibliografía